In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate


import warnings; warnings.simplefilter('ignore')

In [8]:
def baseline_model(metadata_path, links_small_path, title):
    meta = pd.read_csv(metadata_path)
    links_small = pd.read_csv(links_small_path)
    meta['genres'] = meta['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
    meta = meta.drop([19730, 29503, 35587])
    meta['id'] = meta['id'].astype('int')
    smd = meta[meta['id'].isin(links_small)]
    smd['tagline'] = smd['tagline'].fillna('')
    smd['description'] = smd['overview'] + smd['tagline']
    smd['description'] = smd['description'].fillna('')
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(smd['description'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    smd = smd.reset_index()
    titles = smd['title']
    indices = pd.Series(smd.index, index=smd['title'])
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    #return top 10 recommendation movie
    return titles.iloc[movie_indices].head(10)

In [9]:
baseline_model('movies_metadata.csv', 'links_small.csv', 'Forrest Gump')

8435                           Frozen
3764                      Serendipity
353                     Jason's Lyric
655                        Phenomenon
7980                    Perfect Sense
7856                      Project Nim
4425                      City of God
3318    Long Night's Journey Into Day
5040                        Jane Eyre
5439                     The Notebook
Name: title, dtype: object